# Install

In [ ]:
!pip3 install pshmodule

In [ ]:
!pip3 install pickle5

In [ ]:
!pip3 install pandas==1.5.0

In [ ]:
!pip3 install swifter

In [ ]:
!pip3 install openai

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Data Load

In [7]:
import sys
sys.path.append('/content/drive/MyDrive/Advertising_By_Personality/src/business_model/keyword_extraction')
print(sys.path)

['/content', '/env/python', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/usr/local/lib/python3.10/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.10/dist-packages/IPython/extensions', '/root/.ipython', '/content/drive/MyDrive/Advertising_By_Personality/src/business_model/keyword_extraction']


In [8]:
import swifter
import config as cfg
import pandas as pd
from tqdm import tqdm
from pshmodule.utils import filemanager as fm

In [20]:
# df = fm.load("/content/drive/MyDrive/Advertising_By_Personality/data/reports/test.xlsx")
df = fm.load(cfg.origin_data)

extension : .xlsx
Loaded 25001 records from /content/drive/MyDrive/Advertising_By_Personality/data/origin/origin.xlsx


이름 변경

In [21]:
new_header = df.iloc[0]
df = df[1:]
df.columns = new_header

In [22]:
df.rename(columns={'관리번호':'no', '광고 구분':'advertisement_type', '마케팅전략':'marketing_strategy', '일상정보':'daily_information', '시즌정보':'season_information', '분류':'type', '제목':'title', '본문':'content', '납품차수':'degree'}, inplace=True)

In [23]:
df.head()

,no,advertisement_type,marketing_strategy,daily_information,season_information,type,title,content,degree
1,1,NaN,NaN,NaN,NaN,원문,[CJONE VIP] 계절밥상 1만원/5천원 할인쿠폰,"마지막 2일! [계절밥상 디너/주말 1만원, 런치 5천원 할인 쿠폰] 9월30일까지...",NaN
2,1,할인/쿠폰/혜택,"매력적인숫자(시간,돈)",NaN,NaN,ST,★최대 1만원★ 계절밥상 할인쿠폰,"이.틀.만! VIP 고객 디너&주말 1만원, 런치 5천원 할인 쿠폰 놓치지 마세요!",1차
3,1,할인/쿠폰/혜택,핵심정보만 전달,NaN,NaN,SF,ONLY VIP! 할인쿠폰♬,소중한 이들과 더 든든하게 먹고 싶다면?\\[계절밥상] 디너/주말 1만원+런치 5천...,1차
4,1,할인/쿠폰/혜택,호기심유발(질문),NaN,NaN,NT,이 혜택💰 놓치지 않을 거예요,"[계절밥상] 디너/주말 1O,OOO원, 런치 5,OOO원 할인\\9월 30일까지🚨",1차
5,1,할인/쿠폰/혜택,호기심유발(이모지),NaN,NaN,NF,VIP님들만을 위한 쿠폰 도착💌,계절밥상에서 든든한 한끼🍱 어때요?\\런치/디너/주말 할인 쿠폰(~9/30) 사용하...,1차


원문 제거 & 2,500건

In [24]:
df_st = df[df.type.str.contains("ST")][:2500]
df_sf = df[df.type.str.contains("SF")][:2500]

In [25]:
print(df_st.shape)
print(df_sf.shape)

(2500, 9)
(2500, 9)


In [26]:
df = pd.concat([df_st, df_sf])

In [27]:
df.reset_index(drop=True, inplace=True)

In [28]:
df.head()

,no,advertisement_type,marketing_strategy,daily_information,season_information,type,title,content,degree
0,1,할인/쿠폰/혜택,"매력적인숫자(시간,돈)",NaN,NaN,ST,★최대 1만원★ 계절밥상 할인쿠폰,"이.틀.만! VIP 고객 디너&주말 1만원, 런치 5천원 할인 쿠폰 놓치지 마세요!",1차
1,2,기타 이벤트,핵심정보만 전달,NaN,NaN,ST,이벤트 참여하면 1OO% 당첨?!,"2OO만원 상당 여행권, 빕스 쿠폰까지!\\누구나 5OOP만 있다면 선물이 와르르★",1차
2,3,할인/쿠폰/혜택,핵심정보만 전달,NaN,NaN,ST,7일 뒤면 사라지는 쿠폰...❗,7/일/뒤 혜택 종료!! 지금 앱에 접속해서 미사용 쿠폰 체크✔️체크✔️,1차
3,4,할인/쿠폰/혜택,"매력적인숫자(시간,돈)",NaN,주말,ST,[계절밥상] 할인 쿠폰 발견!,딱 일요일까지만 이용 가능! ★1만원★ 할인 받고 통돼지구이 먹으러 가기 ☞,1차
4,5,할인/쿠폰/혜택,시간제한,NaN,NaN,ST,[TAG1]님의 VIP혜택 도착,오/늘/단/하/루 대림미술관 무료 초대 혜택 [VIP FREE PASS]\\바로 사...,1차


In [29]:
print(df.type.value_counts())
print(df.shape)

ST    2500
SF    2500
Name: type, dtype: int64
(5000, 9)


# Preprocessing

In [30]:
from pshmodule.processing import processing as p

In [31]:
nlp = p.Nlp()

In [32]:
df_temp = df.copy()

In [33]:
# \\\\ → \\
df_temp['title_processing'] = df_temp.title.swifter.apply(lambda x: x.replace('\\\\', '\\'))
df_temp['content_processing'] = df_temp.content.swifter.apply(lambda x: x.replace('\\\\', '\\'))

# \n\n -> \\\\
df_temp['title_processing'] = df_temp.title_processing.replace('\n\n', '\\\\').replace('\n', '\\')
df_temp['content_processing'] = df_temp.content_processing.replace('\n\n', '\\\\').replace('\n', '\\')

Pandas Apply:   0%|          | 0/5000 [00:00<?, ?it/s]

In [34]:
df_temp['input'] = df_temp.title_processing + '\\\\' + df_temp.content_processing
df_temp['origin'] = df_temp.title + '\\\\' + df_temp.content

In [35]:
df_temp.head()

,no,advertisement_type,marketing_strategy,daily_information,season_information,type,title,content,degree,title_processing,content_processing,input,origin
0,1,할인/쿠폰/혜택,"매력적인숫자(시간,돈)",NaN,NaN,ST,★최대 1만원★ 계절밥상 할인쿠폰,"이.틀.만! VIP 고객 디너&주말 1만원, 런치 5천원 할인 쿠폰 놓치지 마세요!",1차,★최대 1만원★ 계절밥상 할인쿠폰,"이.틀.만! VIP 고객 디너&주말 1만원, 런치 5천원 할인 쿠폰 놓치지 마세요!","★최대 1만원★ 계절밥상 할인쿠폰\\이.틀.만! VIP 고객 디너&주말 1만원, 런...","★최대 1만원★ 계절밥상 할인쿠폰\\이.틀.만! VIP 고객 디너&주말 1만원, 런..."
1,2,기타 이벤트,핵심정보만 전달,NaN,NaN,ST,이벤트 참여하면 1OO% 당첨?!,"2OO만원 상당 여행권, 빕스 쿠폰까지!\\누구나 5OOP만 있다면 선물이 와르르★",1차,이벤트 참여하면 1OO% 당첨?!,"2OO만원 상당 여행권, 빕스 쿠폰까지!\누구나 5OOP만 있다면 선물이 와르르★","이벤트 참여하면 1OO% 당첨?!\\2OO만원 상당 여행권, 빕스 쿠폰까지!\누구나...","이벤트 참여하면 1OO% 당첨?!\\2OO만원 상당 여행권, 빕스 쿠폰까지!\\누구..."
2,3,할인/쿠폰/혜택,핵심정보만 전달,NaN,NaN,ST,7일 뒤면 사라지는 쿠폰...❗,7/일/뒤 혜택 종료!! 지금 앱에 접속해서 미사용 쿠폰 체크✔️체크✔️,1차,7일 뒤면 사라지는 쿠폰...❗,7/일/뒤 혜택 종료!! 지금 앱에 접속해서 미사용 쿠폰 체크✔️체크✔️,7일 뒤면 사라지는 쿠폰...❗\\7/일/뒤 혜택 종료!! 지금 앱에 접속해서 미사...,7일 뒤면 사라지는 쿠폰...❗\\7/일/뒤 혜택 종료!! 지금 앱에 접속해서 미사...
3,4,할인/쿠폰/혜택,"매력적인숫자(시간,돈)",NaN,주말,ST,[계절밥상] 할인 쿠폰 발견!,딱 일요일까지만 이용 가능! ★1만원★ 할인 받고 통돼지구이 먹으러 가기 ☞,1차,[계절밥상] 할인 쿠폰 발견!,딱 일요일까지만 이용 가능! ★1만원★ 할인 받고 통돼지구이 먹으러 가기 ☞,[계절밥상] 할인 쿠폰 발견!\\딱 일요일까지만 이용 가능! ★1만원★ 할인 받고 ...,[계절밥상] 할인 쿠폰 발견!\\딱 일요일까지만 이용 가능! ★1만원★ 할인 받고 ...
4,5,할인/쿠폰/혜택,시간제한,NaN,NaN,ST,[TAG1]님의 VIP혜택 도착,오/늘/단/하/루 대림미술관 무료 초대 혜택 [VIP FREE PASS]\\바로 사...,1차,[TAG1]님의 VIP혜택 도착,오/늘/단/하/루 대림미술관 무료 초대 혜택 [VIP FREE PASS]\바로 사용...,[TAG1]님의 VIP혜택 도착\\오/늘/단/하/루 대림미술관 무료 초대 혜택 [V...,[TAG1]님의 VIP혜택 도착\\오/늘/단/하/루 대림미술관 무료 초대 혜택 [V...


In [36]:
df_temp = df_temp[['type', 'input']]

In [37]:
df_temp.head()

,type,input
0,ST,"★최대 1만원★ 계절밥상 할인쿠폰\\이.틀.만! VIP 고객 디너&주말 1만원, 런..."
1,ST,"이벤트 참여하면 1OO% 당첨?!\\2OO만원 상당 여행권, 빕스 쿠폰까지!\누구나..."
2,ST,7일 뒤면 사라지는 쿠폰...❗\\7/일/뒤 혜택 종료!! 지금 앱에 접속해서 미사...
3,ST,[계절밥상] 할인 쿠폰 발견!\\딱 일요일까지만 이용 가능! ★1만원★ 할인 받고 ...
4,ST,[TAG1]님의 VIP혜택 도착\\오/늘/단/하/루 대림미술관 무료 초대 혜택 [V...


In [38]:
df_temp.shape

(5000, 2)

# Save

In [40]:
fm.save(cfg.for_gpt, df_temp)

Saved 5000 records


# Pickle to xlsx


### data load

In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/Advertising_By_Personality/src/business_model/keyword_extraction')
print(sys.path)

['/content', '/env/python', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/usr/local/lib/python3.10/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.10/dist-packages/IPython/extensions', '/root/.ipython', '/content/drive/MyDrive/Advertising_By_Personality/src/business_model/keyword_extraction']


In [ ]:
import json
import swifter
import pandas as pd
from pshmodule.utils import filemanager as fm
from pshmodule.processing import processing as p

In [ ]:
nlp = p.Nlp()

In [ ]:
df = fm.load('/content/drive/MyDrive/Advertising_By_Personality/data/train_final.json')

extension : .json
Loaded 2000 records from /content/drive/MyDrive/Advertising_By_Personality/data/train_final.json


In [ ]:
df.head()

,NT,NF,marketing_entity,marketing_target,benefit_conditions,benefits,discount_figure,promotional_items,promotional_place,event_period,dow_information,season_information,solicitation_point,type,label
0,"이 혜택\ud83d\udcb0 놓치지 않을 거예요, [계절밥상] 디너/주말 10,0...",없음,계절밥상,없음,없음,"디너/주말 10,000원, 런치 5,000원 할인","10,000원 디너/주말, 5,000원 런치",계절밥상,없음,9월 30일까지,없음,없음,없음,NT,"이 혜택\ud83d\udcb0 놓치지 않을 거예요\\[계절밥상] 디너/주말 10,0..."
1,"무조건 당첨인데 참여 외않해?\ud83d\udc40, 500P만 있으면 무조건 당첨...",없음,없음,없음,500P,"200만원 상당 여행권, 빕스 쿠폰",없음,없음,없음,없음,없음,없음,없음,NT,무조건 당첨인데 참여 외않해?\ud83d\udc40\\500P만 있으면 무조건 당첨...
2,"아직 손도 안 댄 쿠폰이 있어요!, 미사용 쿠폰 만료까지 D-7\ud83d\udea...",없음,없음,없음,미사용 쿠폰 만료까지 확인,없음,없음,없음,없음,D-7,없음,없음,없음,NT,아직 손도 안 댄 쿠폰이 있어요!\\미사용 쿠폰 만료까지 D-7\ud83d\udea...
3,"주말엔 사라질 그 할인쿠폰!, [0000 1만원 할인쿠폰], 유효기간은 이번 주 일...",없음,없음,없음,할인 쿠폰 사용,1만원 할인,10000원,통돼지구이,없음,이번 주 일요일까지,일요일,없음,없음,NT,주말엔 사라질 그 할인쿠폰!\\[0000 1만원 할인쿠폰] 의 유효기간은 이번 주 ...
4,"득템프 1회 더 찍으면 5,000원 할인 쿠폰도 드린답니다","아참, 누리세오!",누리세오,없음,득템프 1회 더 찍기,"5,000원 할인 쿠폰","5,000원",바캉스 위크,없음,없음,없음,없음,없음,NT,"00 바캉스 위크 특가⚡누리세오!\\아참, 득템프 1회 더 찍으면 5,000원 할인..."


### processing

##### 문구 살리기

In [ ]:
df_nt = df[df.type == 'NT']
df_nf = df[df.type == 'NF']

In [ ]:
df_nt['sent'] = df_nt.NT
df_nf['sent'] = df_nt.NF

<ipython-input-7-a5385d565df3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nt['sent'] = df_nt.NT
<ipython-input-7-a5385d565df3>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nf['sent'] = df_nt.NF


In [ ]:
df_nt = df_nt[['sent', 'marketing_entity', 'marketing_target', 'benefit_conditions', 'benefits', 'discount_figure', 'promotional_items', 'promotional_place', 'event_period', 'dow_information', 'season_information', 'solicitation_point', 'type', 'label']]
df_nf = df_nf[['sent', 'marketing_entity', 'marketing_target', 'benefit_conditions', 'benefits', 'discount_figure', 'promotional_items', 'promotional_place', 'event_period', 'dow_information', 'season_information', 'solicitation_point', 'type', 'label']]

In [ ]:
df = pd.concat([df_nt, df_nf], axis=0)

In [ ]:
df.shape

(2000, 14)

##### 이모지 되돌리기

In [ ]:
df['sent'] = df.sent.swifter.apply(lambda x: nlp.convert_emojis_in_text(nlp.convert_to_python_unicode(str(x))))
df['marketing_entity'] = df.marketing_entity.swifter.apply(lambda x: nlp.convert_emojis_in_text(nlp.convert_to_python_unicode(str(x))))
df['marketing_target'] = df.marketing_target.swifter.apply(lambda x: nlp.convert_emojis_in_text(nlp.convert_to_python_unicode(str(x))))
df['benefit_conditions'] = df.benefit_conditions.swifter.apply(lambda x: nlp.convert_emojis_in_text(nlp.convert_to_python_unicode(str(x))))
df['benefits'] = df.benefits.swifter.apply(lambda x: nlp.convert_emojis_in_text(nlp.convert_to_python_unicode(str(x))))
df['discount_figure'] = df.discount_figure.swifter.apply(lambda x: nlp.convert_emojis_in_text(nlp.convert_to_python_unicode(str(x))))
df['promotional_items'] = df.promotional_items.swifter.apply(lambda x: nlp.convert_emojis_in_text(nlp.convert_to_python_unicode(str(x))))
df['promotional_place'] = df.promotional_place.swifter.apply(lambda x: nlp.convert_emojis_in_text(nlp.convert_to_python_unicode(str(x))))
df['event_period'] = df.event_period.swifter.apply(lambda x: nlp.convert_emojis_in_text(nlp.convert_to_python_unicode(str(x))))
df['dow_information'] = df.dow_information.swifter.apply(lambda x: nlp.convert_emojis_in_text(nlp.convert_to_python_unicode(str(x))))
df['season_information'] = df.season_information.swifter.apply(lambda x: nlp.convert_emojis_in_text(nlp.convert_to_python_unicode(str(x))))
df['solicitation_point'] = df.solicitation_point.swifter.apply(lambda x: nlp.convert_emojis_in_text(nlp.convert_to_python_unicode(str(x))))
df['label'] = df.label.swifter.apply(lambda x: nlp.convert_emojis_in_text(nlp.convert_to_python_unicode(str(x))))

Pandas Apply:   0%|          | 0/2000 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/2000 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/2000 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/2000 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/2000 [00:00<?, ?it/s]

...

Pandas Apply:   0%|          | 0/2000 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/2000 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/2000 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/2000 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/2000 [00:00<?, ?it/s]

In [ ]:
df['sent'] = df.sent.swifter.apply(lambda x: x.encode('utf-8', 'ignore').decode('utf-8'))
df['label'] = df.label.swifter.apply(lambda x: x.encode('utf-8', 'ignore').decode('utf-8'))

Pandas Apply:   0%|          | 0/2000 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/2000 [00:00<?, ?it/s]

In [ ]:
df['sent'] = df.sent.swifter.apply(lambda x: x.replace('\\\\', '\n\n'))
df['sent'] = df.sent.swifter.apply(lambda x: x.replace('\\', '\n'))
df['label'] = df.label.swifter.apply(lambda x: x.replace('\\\\', '\n\n'))
df['label'] = df.label.swifter.apply(lambda x: x.replace('\\', '\n'))

Pandas Apply:   0%|          | 0/2000 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/2000 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/2000 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/2000 [00:00<?, ?it/s]

In [ ]:
df.head()

,sent,marketing_entity,marketing_target,benefit_conditions,benefits,discount_figure,promotional_items,promotional_place,event_period,dow_information,season_information,solicitation_point,type,label
0,"이 혜택💰 놓치지 않을 거예요, [계절밥상] 디너/주말 10,000원, 런치 5,0...",계절밥상,없음,없음,"디너/주말 10,000원, 런치 5,000원 할인","10,000원 디너/주말, 5,000원 런치",계절밥상,없음,9월 30일까지,없음,없음,없음,NT,"이 혜택💰 놓치지 않을 거예요\n\n[계절밥상] 디너/주말 10,000원, 런치 5..."
1,"무조건 당첨인데 참여 외않해?👀, 500P만 있으면 무조건 당첨되는 갓-이벤트✨, ...",없음,없음,500P,"200만원 상당 여행권, 빕스 쿠폰",없음,없음,없음,없음,없음,없음,없음,NT,무조건 당첨인데 참여 외않해?👀\n\n500P만 있으면 무조건 당첨되는 갓-이벤트✨...
2,"아직 손도 안 댄 쿠폰이 있어요!, 미사용 쿠폰 만료까지 D-7🚨 쿠폰함으로 확인하...",없음,없음,미사용 쿠폰 만료까지 확인,없음,없음,없음,없음,D-7,없음,없음,없음,NT,아직 손도 안 댄 쿠폰이 있어요!\n\n미사용 쿠폰 만료까지 D-7🚨 쿠폰함으로 확...
3,"주말엔 사라질 그 할인쿠폰!, [0000 1만원 할인쿠폰], 유효기간은 이번 주 일...",없음,없음,할인 쿠폰 사용,1만원 할인,10000원,통돼지구이,없음,이번 주 일요일까지,일요일,없음,없음,NT,주말엔 사라질 그 할인쿠폰!\n\n[0000 1만원 할인쿠폰] 의 유효기간은 이번 ...
4,"득템프 1회 더 찍으면 5,000원 할인 쿠폰도 드린답니다",누리세오,없음,득템프 1회 더 찍기,"5,000원 할인 쿠폰","5,000원",바캉스 위크,없음,없음,없음,없음,없음,NT,"00 바캉스 위크 특가⚡누리세오!\n\n아참, 득템프 1회 더 찍으면 5,000원 ..."


### 이름 변경

In [ ]:
df.rename(columns={'sent':'MBTI 성향 문구', 'marketing_entity':'마케팅 주체', 'marketing_target':'마케팅 대상', 'benefit_conditions':'혜택 지급 조건', 'benefits':'혜택', 'discount_figure':'할인 수치', 'promotional_items':'프로모션 품목', 'promotional_place':'프로모션 장소', 'event_period':'이벤트 기간', 'dow_information':'요일 정보', 'season_information':'시즌 정보', 'solicitation_point':'소구점', 'type':'구분', 'label':'cj 원본 광고문구'}, inplace=True)

In [ ]:
df.head()

,MBTI 성향 문구,마케팅 주체,마케팅 대상,혜택 지급 조건,혜택,할인 수치,프로모션 품목,프로모션 장소,이벤트 기간,요일 정보,시즌 정보,소구점,구분,cj 원본 광고문구
0,"이 혜택💰 놓치지 않을 거예요, [계절밥상] 디너/주말 10,000원, 런치 5,0...",계절밥상,없음,없음,"디너/주말 10,000원, 런치 5,000원 할인","10,000원 디너/주말, 5,000원 런치",계절밥상,없음,9월 30일까지,없음,없음,없음,NT,"이 혜택💰 놓치지 않을 거예요\n\n[계절밥상] 디너/주말 10,000원, 런치 5..."
1,"무조건 당첨인데 참여 외않해?👀, 500P만 있으면 무조건 당첨되는 갓-이벤트✨, ...",없음,없음,500P,"200만원 상당 여행권, 빕스 쿠폰",없음,없음,없음,없음,없음,없음,없음,NT,무조건 당첨인데 참여 외않해?👀\n\n500P만 있으면 무조건 당첨되는 갓-이벤트✨...
2,"아직 손도 안 댄 쿠폰이 있어요!, 미사용 쿠폰 만료까지 D-7🚨 쿠폰함으로 확인하...",없음,없음,미사용 쿠폰 만료까지 확인,없음,없음,없음,없음,D-7,없음,없음,없음,NT,아직 손도 안 댄 쿠폰이 있어요!\n\n미사용 쿠폰 만료까지 D-7🚨 쿠폰함으로 확...
3,"주말엔 사라질 그 할인쿠폰!, [0000 1만원 할인쿠폰], 유효기간은 이번 주 일...",없음,없음,할인 쿠폰 사용,1만원 할인,10000원,통돼지구이,없음,이번 주 일요일까지,일요일,없음,없음,NT,주말엔 사라질 그 할인쿠폰!\n\n[0000 1만원 할인쿠폰] 의 유효기간은 이번 ...
4,"득템프 1회 더 찍으면 5,000원 할인 쿠폰도 드린답니다",누리세오,없음,득템프 1회 더 찍기,"5,000원 할인 쿠폰","5,000원",바캉스 위크,없음,없음,없음,없음,없음,NT,"00 바캉스 위크 특가⚡누리세오!\n\n아참, 득템프 1회 더 찍으면 5,000원 ..."


### 파일 저장

In [ ]:
!ls /content/drive/MyDrive/Advertising_By_Personality/data/reports

 dataset.xlsx   predict.xlsx   test.xlsx  '모델 결과.gsheet'


In [ ]:
df.to_excel('/content/drive/MyDrive/Advertising_By_Personality/data/reports/dataset.xlsx', index = False, encoding = "utf-8")